In [1]:
# The Helmholtz equation

import sys

import numpy as np
from mpi4py import MPI
from petsc4py import PETSc

import ufl
from dolfinx.fem import Function, FunctionSpace
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import XDMFFile, VTXWriter
from dolfinx.io.gmshio import model_to_mesh


if not np.issubdtype(PETSc.ScalarType, np.complexfloating):
    print("This tutorial requires complex number support")
    exit(0)
else:
    print(f"Using {PETSc.ScalarType}.")

Using <class 'numpy.complex128'>.


In [2]:
try:
    import gmsh
except ModuleNotFoundError:
    print("This demo requires gmsh to be installed")
    sys.exit(0)

from mesh_generation import generate_mesh

k0 = 10 * np.pi
lmbda = 2 * np.pi / k0

model = generate_mesh(lmbda, 2)
mesh, cell_tags, facet_tags = model_to_mesh(model, MPI.COMM_WORLD, 0, gdim=2)
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 70%] Meshing curve 5 (Circle)
Info    : [ 90%] Meshing curve 6 (Circle)
Info    : Done meshing 1D (Wall 0.000980396s, CPU 0.001561s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0465658s, CPU 0.045643s)
Info    : 1627 nodes 3286 elements


Error   : Unknown mesh format '/root/.local/share/jupyter/runtime/kernel-f4c697f2-e3af-41ca-bd67-7ed16179a5e5.json'


In [6]:
n = ufl.FacetNormal(mesh)

p = 4

# Definition of function space
element = ufl.FiniteElement("Lagrange", ufl.triangle, p)
V = FunctionSpace(mesh, element)


# Define wave number
DG = FunctionSpace(mesh, ("DG", 0))
k = Function(DG)
k.x.array[:] = k0
k.x.array[cell_tags.values == 1] = 2*k0

x = ufl.SpatialCoordinate(mesh)
ui = ufl.exp(1.0j * k * x[0])
g = ufl.dot(ufl.grad(ui), n) + 1j * k * ui

# Define variational problem
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Weak Form
ds = ufl.Measure("ds", domain=mesh)
dx = ufl.Measure("dx", domain=mesh)
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * dx \
    - k**2 * ufl.inner(u, v) * dx \
    + 1j * k * ufl.inner(u, v) * ds
L = ufl.inner(g, v) * ds


# Compute solution
uh = Function(V)
uh.name = "u"
problem = LinearProblem(a, L, u=uh, petsc_options={
                        "ksp_type": "preonly", "pc_type": "lu"})
problem.solve()

# XDMF write the solution as a P1 function
with XDMFFile(MPI.COMM_WORLD, "out.xdmf", "w") as file:
    file.write_mesh(mesh)
    file.write_function(uh)

# VTX can write higher order function
with VTXWriter(MPI.COMM_WORLD, "out_high_order.bp", uh) as f:
    f.write(0.0)